In [1]:
# this is some change in this code

In [1]:
# this is my second change

In [ ]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [8]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [33]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [15]:
lr = LinearRegression(featuresCol='features',labelCol='label',
                     predictionCol='prediction')

In [16]:
lrModel = lr.fit(training)

In [18]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [19]:
lrModel.intercept

0.14228558260358093

### summary attribute

In [20]:
training_summary = lrModel.summary

In [21]:
training_summary.r2

0.027839179518600154

In [22]:
training_summary.rootMeanSquaredError

10.16309157133015

### split data into train and test set

In [23]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [29]:
train_data,test_data= all_data.randomSplit([0.7,0.3])

In [31]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                351|
|   mean|0.29436685284943825|
| stddev|  10.52633127982159|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [34]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                150|
|   mean|0.16919023046359155|
| stddev|  9.846622828357399|
|    min|-28.046018037776633|
|    max| 22.923352376063196|
+-------+-------------------+



In [35]:
correct_model = lr.fit(train_data)

In [39]:
# check how well the model work on test data
test_results = correct_model.evaluate(test_data)

In [40]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-28.083098194547354|
| -21.42357618254789|
|  -19.5089029995153|
|-17.829802838211716|
|-14.446938558140829|
|-17.351774972711272|
| -16.38951967330329|
|-18.798143927430953|
| -17.12780368155926|
| -14.77078332101049|
|-11.278784722101676|
|-16.795450769705266|
| -7.947378496079858|
|-13.009856335183711|
|-13.110351842224295|
|-12.878229724868053|
|-10.446965768142599|
|-14.199146579722253|
| -7.699268176561965|
|-13.512267397277228|
+-------------------+
only showing top 20 rows



In [41]:
test_results.rootMeanSquaredError

9.812481154834575

### no lable data

In [42]:
unlabeled_data = test_data.select('features')

In [43]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [44]:
predictions = correct_model.transform(unlabeled_data)

In [45]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| 0.03708015677072139|
|(10,[0,1,2,3,4,5,...|   -2.08730790777508|
|(10,[0,1,2,3,4,5,...|  1.2336894335106696|
|(10,[0,1,2,3,4,5,...| 0.02617664954719973|
|(10,[0,1,2,3,4,5,...| -3.0472617987425146|
|(10,[0,1,2,3,4,5,...|0.025054240035324297|
|(10,[0,1,2,3,4,5,...|  0.3038606322817999|
|(10,[0,1,2,3,4,5,...|    3.01745889480765|
|(10,[0,1,2,3,4,5,...|   1.395715409320014|
|(10,[0,1,2,3,4,5,...| -0.6666014724207273|
|(10,[0,1,2,3,4,5,...| -1.8745508842638545|
|(10,[0,1,2,3,4,5,...|   3.873227666334844|
|(10,[0,1,2,3,4,5,...|  -4.825848503171339|
|(10,[0,1,2,3,4,5,...|  0.4512805463275226|
|(10,[0,1,2,3,4,5,...|  0.6310716307727982|
|(10,[0,1,2,3,4,5,...|   0.783878446332794|
|(10,[0,1,2,3,4,5,...|  -1.410384597286826|
|(10,[0,1,2,3,4,5,...|   2.372073583329682|
|(10,[0,1,2,3,4,5,...|  -3.941281501326861|
|(10,[0,1,2,3,4,5,...|  1.896492